# Crear solicitudes de servicio

Configuramos un controlador para crear solicitudes de servicio en formato JSON con mayor agilidad a traves del kernel.

In [1]:
import pandas as pd
import certifi
import pymongo as py
import datetime
import bson
import os

In [2]:
# Creamos una variable con la conexion a la base de datos

from pymongo import MongoClient
from dotenv import load_dotenv # Get env variables
load_dotenv()

mongo_uri = os.getenv('MONGO_URI')
cartonadoradores = MongoClient(mongo_uri)


# Seleccionamos la BD previamente creada en mongo shell y con las validaciones ya configuradas.

df = cartonadoradores['cartonadoradores']

# Importamos el metodo para introducir objectId directamente.

from bson import ObjectId

### Validación del esquema de validación en MongoDB

![image.png](../recursos/validacionSolicitudesServicio.png)

In [35]:
# Función para introducir solicitudes a la BD. La validación del tipo de dato esta configurada en la BD. 

def crearsolicitudServicio(idUsuario, tipo, estado, foliointerno: str, idServicio, idLocal, descripcion, importe: float, fechaEmision, fechaCierre):
    df.solicitudesServicios.insert_one({
    "idUsuario": ObjectId(idUsuario),
    "tipo": tipo,
    "estado": estado,
    "foliointerno": foliointerno,
    "Servicios": [{"idServicio" : ObjectId(idServicio),
                "idLocal" : idLocal,
                "descripcion": descripcion,
                "importe": importe
                }],
    "fechaEmision": datetime.datetime.strptime(fechaEmision, '%Y-%m-%d'),
    "fechaCierre": datetime.datetime.strptime(fechaCierre, '%Y-%m-%d')
    })
    
# Como una orden puede tener más de un servicio, creamos una función para añadir uno.
    
def agregarServicio(idOrden, idServicio, idLocal, descripcion, importe: float):
    df.solicitudesServicios.update_one({"_id": ObjectId(idOrden)}, {'$push': {"Servicios": {
        "idServicio" : ObjectId(idServicio),
        "idLocal" : idLocal,
        "descripcion": descripcion,
        "importe": importe
    }}})

In [43]:
# Corremos la funcion para agregar solicitudes.

crearsolicitudServicio('5fbb61187757f4349a0027bd',\
             'ordenProduccion', \
             'abierta', \
             '0000000031', \
                '5fbb61187757f4349a0027bd',\
                'idServicio',\
                'piezas',\
                 30375.01,\
             '2020-09-09',\
             '2020-09-10')

In [39]:
# Corremos la funcion para agregar un servicio.

agregarServicio('5fc2ba0f63a03f8681500703','5fc2bbb804237895cbc61541',\
                'jojo',\
                'red riot',\
                 58.1)